In [3]:
from transformers import pipeline
import json

# Initialize zero-shot classification pipelines for BERT and RoBERTa
bert_classifier = pipeline("zero-shot-classification", model="bert-base-uncased")
roberta_classifier = pipeline("zero-shot-classification", model="roberta-base")

# Sample text
text = "I'm having trouble with my internet connection."

# Candidate labels (intents)
candidate_labels = ["Technical Support", "Billing Inquiry", "Service Complaint", "General Inquiry"]

# Perform zero-shot classification with BERT
bert_predictions = bert_classifier(text, candidate_labels)

# Perform zero-shot classification with RoBERTa
roberta_predictions = roberta_classifier(text, candidate_labels)

# Print results
print("BERT Predictions:")
print(json.dumps(bert_predictions,indent=4))

print("\nRoBERTa Predictions:")
print(json.dumps(roberta_predictions,indent=4))

# Get the best prediction for BERT
best_bert_prediction = max(bert_predictions["scores"])
best_bert_label = bert_predictions["labels"][bert_predictions["scores"].index(best_bert_prediction)]

# Get the best prediction for RoBERTa
best_roberta_prediction = max(roberta_predictions["scores"])
best_roberta_label = roberta_predictions["labels"][roberta_predictions["scores"].index(best_roberta_prediction)]

# Print the best predictions
print("Best BERT Prediction:", best_bert_label)
print("Best RoBERTa Prediction:", best_roberta_label)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model co

BERT Predictions:
{
    "sequence": "I'm having trouble with my internet connection.",
    "labels": [
        "General Inquiry",
        "Billing Inquiry",
        "Service Complaint",
        "Technical Support"
    ],
    "scores": [
        0.33063021302223206,
        0.23186515271663666,
        0.22003042697906494,
        0.21747422218322754
    ]
}

RoBERTa Predictions:
{
    "sequence": "I'm having trouble with my internet connection.",
    "labels": [
        "Technical Support",
        "Service Complaint",
        "Billing Inquiry",
        "General Inquiry"
    ],
    "scores": [
        0.2516486644744873,
        0.2500995695590973,
        0.24931930005550385,
        0.24893245100975037
    ]
}
Best BERT Prediction: General Inquiry
Best RoBERTa Prediction: Technical Support


In [5]:
import time
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import warnings

# Suppress the specific UserWarning
warnings.filterwarnings("ignore", message="Length of IterableDataset")

# Load the dataset from CSV
dataset = pd.read_csv("customer_support_tickets.csv")
# Load the dataset from CSV


# Select a subset of 100 samples
subset = dataset.sample(n=100, random_state=42)

# Define the models
models = {
    "BERT": "bert-base-uncased",
    "RoBERTa": "roberta-base",
    "SBERT": "sentence-transformers/paraphrase-MiniLM-L6-v2"
}

# Initialize an empty DataFrame to store metrics
metrics_df = pd.DataFrame(columns=["Model", "Accuracy", "Precision", "Recall", "F1-score", "Time taken"])

# Iterate over models
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")
    
    # Initialize zero-shot classification pipeline
    classifier = pipeline("zero-shot-classification", model=model)
    
    # Extract ticket descriptions and true labels from the subset
    texts = subset["Ticket Description"].tolist()
    true_labels = subset["Ticket Type"].tolist()
    
    # Candidate labels (intents) - assuming unique ticket types are the labels
    candidate_labels = subset["Ticket Type"].unique().tolist()
    
    # Measure the time taken for classification
    start_time = time.time()
    
    # Perform zero-shot classification
    predictions = classifier(texts, candidate_labels=candidate_labels, multi_label=False)
    
    # Measure the time taken for classification
    end_time = time.time()
    duration = end_time - start_time
    
    # Extract predicted labels
    predicted_labels = [prediction['labels'][0] for prediction in predictions]
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    
    # Append metrics to DataFrame
    metrics_df = metrics_df.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Time taken": duration
    }, ignore_index=True)

# Print the metrics DataFrame
print(metrics_df)


Training and evaluating BERT...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
/Users/monu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training and evaluating RoBERTa...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
/Users/monu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training and evaluating SBERT...


...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


     Model  Accuracy  Precision  Recall  F1-score  Time taken
0     BERT      0.17   0.193575    0.17  0.087747   59.506135
1  RoBERTa      0.21   0.086684    0.21  0.119684   63.319368
2    SBERT      0.24   0.243208    0.24  0.207631    8.992744


/Users/monu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
